In [1]:
# %pip install gensim
# %pip install nltk
# %pip install datasets
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import nltk, datasets
from nltk.tokenize import word_tokenize
from datasets import load_dataset
nltk.download('punkt')

c:\Users\dylan\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Error loading punkt: <urlopen error [WinError 10054] An
[nltk_data]     existing connection was forcibly closed by the remote
[nltk_data]     host>


False

In [2]:
import multiprocessing
multiprocessing.cpu_count()

16

In [21]:
dataset = load_dataset('csv', data_files='train.csv', column_names=['_', 'q', 'a'])



Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 500.04it/s]







































































Generating train split: 1400000 examples [00:10, 137029.03 examples/s]


In [22]:
dataset['train'].info

DatasetInfo(description='', citation='', homepage='', license='', features={'_': Value(dtype='string', id=None), 'q': Value(dtype='string', id=None), 'a': Value(dtype='string', id=None), '__index_level_0__': Value(dtype='int64', id=None)}, post_processed=None, supervised_keys=None, task_templates=None, builder_name='csv', dataset_name='csv', config_name='default', version=0.0.0, splits={'train': SplitInfo(name='train', num_bytes=755026740, num_examples=1400000, shard_lengths=[960000, 440000], dataset_name='csv')}, download_checksums={'C:/Users/dylan/OneDrive/Desktop/LING400/train.csv': {'num_bytes': 745966378, 'checksum': None}}, download_size=745966378, post_processing_size=None, dataset_size=755026740, size_in_bytes=1500993118)

In [23]:
dataset = dataset.select_columns(['a'])['train']

In [24]:
dataset

Dataset({
    features: ['a'],
    num_rows: 1400000
})

In [25]:
testTrain = dataset.shard(num_shards=10, index=0)

In [30]:
tagged_data = []
def process(data, idx):
  if data['a']:
    tagged_data.append(TaggedDocument(words=word_tokenize(data['a'].lower()), tags=[str(idx)]))

In [31]:
testTrain.map(process, with_indices=True)

Map: 100%|██████████| 140000/140000 [00:54<00:00, 2575.41 examples/s]


Dataset({
    features: ['a'],
    num_rows: 140000
})

In [33]:
# tagged_data = [data['normalizedBody'] for data in tagged_data.to_list()]
print(len(tagged_data))
type(tagged_data[0])

137512


gensim.models.doc2vec.TaggedDocument

In [35]:
# max_epochs = 100
vec_size = 50
alpha = 0.025

model = Doc2Vec(vector_size=vec_size,
                alpha=alpha,
                min_alpha=0.00025,
                min_count=2,
                dm =1,
                workers=10,
                epochs = 30)

model.build_vocab(tagged_data)

# for epoch in range(max_epochs):
    # print('iteration {0}'.format(epoch))
# model.train(tagged_data,
#             total_examples=model.corpus_count,
#             epochs=model.epochs)
#     # decrease the learning rate
#     # model.alpha -= 0.0002
#     # fix the learning rate, no decay
#     # model.min_alpha = model.alpha

# model.save("d2v.model")
# print("Model Saved")

In [36]:
model.train(tagged_data,
          total_examples=model.corpus_count,
          epochs=model.epochs)


model.save("d2v.model")
print("Model Saved")

Model Saved


In [42]:
ranks = []
second_ranks = []
for doc_id in range(len(tagged_data)):
    inferred_vector = model.infer_vector(tagged_data[doc_id].words)
    sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
    rank = [docid for docid, sim in sims].index(str(doc_id))
    ranks.append(rank)

    second_ranks.append(sims[1])

In [43]:
import collections

counter = collections.Counter(ranks)
print(counter)

Counter({0: 76911, 1: 43, 2: 6, 3: 3, 50: 1, 10887: 1, 4: 1, 6: 1})


In [37]:
from gensim.models.doc2vec import Doc2Vec

model= Doc2Vec.load("d2v.model")
#to find the vector of a document which is not in training data
test_data = word_tokenize("I speak english, and live in america. 7-9, and 11 don't matter much. 13 who cares? 12: if the handful of apps are what I want thats better than 250,000 i need to sort through to find what I want (more likely in apples favor but quantity &lt; quality). 5: would never use either as a phone (I'm not that much of a jackass). 6: Screen size (more a personal preference, get whichever fits your needs. So, we have again the issue of resolution (kindle), speakers (possibly kindle), wifi (I don't know enough, give it to apple), and video compatibility (Its better that apple is compatible, but maybe it doesn't matter, depends on what you want to watch or whether you'd put up with converting the video yourself). TL;DR - Most of the points probably don't matter that much. Get want better suites your needs.".lower())
v1 = model.infer_vector(test_data)
print("V1_infer", v1)

# to find most similar doc using tags
similar_doc = model.dv.most_similar([v1])
print(similar_doc)
print(model.dv[similar_doc[0][0]])

V1_infer [ 0.41704875  0.9128236  -0.77358896 -1.39118    -0.6378415   0.9135672
 -0.537952   -0.5534967  -2.362011   -1.1828707  -2.8130705   0.23066439
  1.4893003  -2.1734345   0.7703471   2.917018    0.10161806 -2.5434446
 -0.49882433  1.0268993   1.3930393  -0.6734877   0.0292497  -0.37665358
 -1.5462773   0.89492065 -0.48627555 -1.1332024   0.02221628  2.2027252
  0.939353    1.1180423   0.953894    0.4733165   0.34838372 -0.04766769
 -1.0460433   0.15472355  0.9235793  -1.840209    0.7940906   0.24389037
  1.8758005   0.9713099   1.0804483  -1.1509734  -1.0200976  -0.8353447
  0.85547197 -0.7747868 ]
[('42843', 0.7511810064315796), ('30491', 0.7235507369041443), ('21513', 0.7169120907783508), ('53583', 0.7014052271842957), ('70731', 0.6993623375892639), ('46880', 0.6926173567771912), ('5421', 0.6795118451118469), ('52077', 0.6677392721176147), ('57345', 0.664455235004425), ('1146', 0.6632039546966553)]
[-0.80004585  1.8123759  -0.7500062  -1.4100467  -0.13876641 -0.14455152
  1.

In [38]:
testTrain['normalizedBody'][int(similar_doc[3][0])]

'1) What is your budget? $600 and below. The lower, the better. \n2) What size notebook would you prefer? 14""-15"" \n3) Where will you buying this notebook? U.S. Online. Where evers cheap \n4) Are there any brands that you prefer or any you really don\'t like? I want a Linux prebuilt, so I hear system76 is good with that. I also like acer and asus and lenovo (some) and I hate Dell, Hp, Sony, Toshiba and Macs \n5) Would you consider laptops that are refurbished/redistributed? Indeed \n 6) What are the primary tasks will you be performing with this notebook?   Hacking, network penetration, coding and scripting, debugging, network maintenance and use it as an IT laptop. \n7) Will you be taking the notebook with you to different places, leaving it on your desk or both? Yes, It will go anywhere I go, but I can handle any weight. \n8) Will you be playing games on your notebook? If so, please state which games or types of games? I have a tower so not much. probably TF 2 and maybe some RPG. I